In [1]:
%%bash
# pip install relevant python packages
pip install -q pycocotools split-folders

## Mount Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Converting Annotation Format

In [3]:
import tqdm
import numpy as np

In [4]:
from pycocotools.coco import COCO
_annotation_path = '/content/drive/MyDrive/trash_segmentation/TACO/data/annotations.json'
data_source = COCO(annotation_file=_annotation_path)

loading annotations into memory...
Done (t=1.50s)
creating index...
index created!


In [5]:
catIds = data_source.getCatIds()
categories = data_source.loadCats(catIds)
categories.sort(key=lambda x: x['id'])
categories

[{'supercategory': 'Aluminium foil', 'id': 0, 'name': 'Aluminium foil'},
 {'supercategory': 'Battery', 'id': 1, 'name': 'Battery'},
 {'supercategory': 'Blister pack', 'id': 2, 'name': 'Aluminium blister pack'},
 {'supercategory': 'Blister pack', 'id': 3, 'name': 'Carded blister pack'},
 {'supercategory': 'Bottle', 'id': 4, 'name': 'Other plastic bottle'},
 {'supercategory': 'Bottle', 'id': 5, 'name': 'Clear plastic bottle'},
 {'supercategory': 'Bottle', 'id': 6, 'name': 'Glass bottle'},
 {'supercategory': 'Bottle cap', 'id': 7, 'name': 'Plastic bottle cap'},
 {'supercategory': 'Bottle cap', 'id': 8, 'name': 'Metal bottle cap'},
 {'supercategory': 'Broken glass', 'id': 9, 'name': 'Broken glass'},
 {'supercategory': 'Can', 'id': 10, 'name': 'Food Can'},
 {'supercategory': 'Can', 'id': 11, 'name': 'Aerosol'},
 {'supercategory': 'Can', 'id': 12, 'name': 'Drink can'},
 {'supercategory': 'Carton', 'id': 13, 'name': 'Toilet tube'},
 {'supercategory': 'Carton', 'id': 14, 'name': 'Other carton'

In [6]:
# creating superclass dictionary
super = [c['supercategory'] for c in categories]
super = sorted(list(set(super))) # removing duplicates

classes = {s: idx for idx, s in enumerate(super)} # class to idx map
classes

{'Aluminium foil': 0,
 'Battery': 1,
 'Blister pack': 2,
 'Bottle': 3,
 'Bottle cap': 4,
 'Broken glass': 5,
 'Can': 6,
 'Carton': 7,
 'Cigarette': 8,
 'Cup': 9,
 'Food waste': 10,
 'Glass jar': 11,
 'Lid': 12,
 'Other plastic': 13,
 'Paper': 14,
 'Paper bag': 15,
 'Plastic bag & wrapper': 16,
 'Plastic container': 17,
 'Plastic glooves': 18,
 'Plastic utensils': 19,
 'Pop tab': 20,
 'Rope & strings': 21,
 'Scrap metal': 22,
 'Shoe': 23,
 'Squeezable tube': 24,
 'Straw': 25,
 'Styrofoam piece': 26,
 'Unlabeled litter': 27}

In [7]:
labels_transfer = {}

for c in categories:
    labels_transfer[c['id']] = classes[c['supercategory']]
print(labels_transfer)

{0: 0, 1: 1, 2: 2, 3: 2, 4: 3, 5: 3, 6: 3, 7: 4, 8: 4, 9: 5, 10: 6, 11: 6, 12: 6, 13: 7, 14: 7, 15: 7, 16: 7, 17: 7, 18: 7, 19: 7, 20: 9, 21: 9, 22: 9, 23: 9, 24: 9, 25: 10, 26: 11, 27: 12, 28: 12, 29: 13, 30: 14, 31: 14, 32: 14, 33: 14, 34: 15, 35: 15, 36: 16, 37: 16, 38: 16, 39: 16, 40: 16, 41: 16, 42: 16, 43: 17, 44: 17, 45: 17, 46: 17, 47: 17, 48: 18, 49: 19, 50: 20, 51: 21, 52: 22, 53: 23, 54: 24, 55: 25, 56: 25, 57: 26, 58: 27, 59: 8}


In [8]:
!mkdir -p tmp/labels tmp/images
!ls

drive  sample_data  tmp


In [9]:
import shutil
import os
IMAGES_PATH = 'tmp/images'
LABELS_PATH = 'tmp/labels/'

In [10]:
img_ids = data_source.getImgIds()

for index, img_id in tqdm.tqdm(enumerate(img_ids), total=len(img_ids)):
    img_info = data_source.loadImgs(img_id)[0]
    
    # img_dir: batch_x/.....jpg ---> batch_x_......jpg
    img_dir = img_info['file_name'].replace('/', '_')
    
    image_name = img_dir.split('.')[0]
    label_dir = LABELS_PATH + image_name + '.txt'
    
    height = img_info['height']
    width = img_info['width']

    # get images
    shutil.copy(f"/content/drive/MyDrive/trash_segmentation/TACO/data/{img_info['file_name']}", os.path.join(IMAGES_PATH, img_dir))
    # shutil.copy(f"/kaggle/input/tacotrashdataset/data/{img_info['file_name']}", os.path.join(IMAGES_PATH, img_dir))
    
    # get labels
    with open(label_dir, mode='w') as fp:
        annotation_id = data_source.getAnnIds(img_id)
        if len(annotation_id) == 0:
            fp.write('')
            continue
        boxes = np.zeros((0, 5))
        annotations = data_source.loadAnns(annotation_id)
        lines = ''
        for annotation in annotations:
            label = labels_transfer[annotation['category_id']]
            box = annotation['bbox']
            # some annotations have basically no width / height (extremely small), skip them
            if box[2] < 1 or box[3] < 1:
                continue
            # top_x,top_y,width,height ----> cen_x,cen_y,width,height
            # standardize to 0-1
            box[0] = round((box[0] + box[2] / 2) / width, 6)
            box[1] = round((box[1] + box[3] / 2) / height, 6)
            box[2] = round(box[2] / width, 6)
            box[3] = round(box[3] / height, 6)
            # line: label x_center y_center width height
            lines = lines + str(label)
            for i in box:
                lines += ' ' + str(i)
            lines += '\n'
        fp.writelines(lines)

100%|██████████| 1500/1500 [07:09<00:00,  3.49it/s]


In [11]:
print(len(os.listdir(IMAGES_PATH)))
print(len(os.listdir(LABELS_PATH)))

1500
1500


## Data Partition

Split data into train, val, test sets respectively.

In [12]:
import splitfolders
_input_folder = 'tmp'
_output_folder = 'tmp2'
splitfolders.ratio(_input_folder, output=_output_folder, seed=42, ratio=(.8, .1, .1), move='false')

Copying files: 3000 files [00:00, 9533.41 files/s]


In [13]:
!cp -R tmp2 /content/drive/MyDrive/trash_segmentation/processed